# Using SFTTrainer evaluate

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re
from datasets import load_dataset, Dataset
import pandas as pd
import os
from tqdm import tqdm
from evaluate import load

# Set CUDA device
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

from evaluate import load


# Load metrics
rouge = load('rouge')
bleu = load('bleu')
meteor = load('meteor')
bertscore = load('bertscore')

MAX_LEN = 2048
BATCH_SIZE=8
# torch_dtype = torch.float32
torch_dtype = torch.bfloat16

model_path = "BounharAbdelaziz/Qwen2.5-0.5B-Instruct-bs-2-lr-2e-05-ep-3-wp-0.1-gacc-16-gnm-1.0-FP16-SFT-mx-2048-v2"
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"


# Load dataset
eval_dataset = load_dataset("BounharAbdelaziz/Arabic-Synthetic-Summarization-Dataset-Filtered", split='validation')
# eval_dataset = eval_dataset.select(range(10))

# Summarize dataset

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_path).eval()#, torch_dtype=torch_dtype)
tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenizer.padding_side='left'
# Set reasonable default for models without max length
tokenizer.model_max_length = MAX_LEN

# Set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
    
# Set chat template
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE
    
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak. 
    This is a workaround to avoid storing too many tensors that are not needed. 
    Taken from https://discuss.huggingface.co/t/cuda-out-of-memory-when-using-trainer-with-compute-metrics/2941/22
    """
    
    # Handle tuple logits (happens when the model is trained using LoRA)
    if isinstance(logits, tuple):
        logits = logits[1]          # logits[0] is the loss value and logits[1] are the logits used to compute loss
                                    # logits: (tensor(2.0426, device='cuda:0'), tensor([[[ 7.8750,  5.3750,  7.0938,  ..., -4.2500, -4.2500, -4.2500],
                                    #          [ 5.0938,  5.0625,  7.3750,  ..., -1.5312, -1.5312, -1.5312],
                                    #          [ 2.6562, -0.9609,  0.0728,  ..., -2.0312, -2.0312, -2.0312],
                                    #          ...,
                                    #          [ 4.1562,  1.4375, -3.6250,  ..., -2.1250, -2.1250, -2.1250],
                                    #          [ 3.7344, -1.6641, -3.8125,  ..., -1.9688, -1.9688, -1.9688],
                                    #          [ 8.1875, -1.2344, -1.6094,  ..., -3.0938, -3.0938, -3.0938]]],
                                    #        device='cuda:0'))

    # Proceed with argmax
    pred_ids = torch.argmax(logits, dim=-1)

    return pred_ids

@torch.no_grad()
def compute_metrics_causal_lm(eval_pred, tokenizer):
    """Compute ROUGE and BLEU scores for evaluation."""
    predictions, references = eval_pred

    # Ensure predictions and references are in the correct format (list or numpy)
    if isinstance(predictions, torch.Tensor):
        predictions = predictions.cpu().numpy()
    if isinstance(references, torch.Tensor):
        references = references.cpu().numpy()

    # Clip token IDs to the valid range
    vocab_size = tokenizer.vocab_size

    def clip_token_ids(token_ids):
        """Clip token IDs to the valid range [0, vocab_size - 1]."""
        # Ensure token_ids is a list or array and clip each value
        return [min(max(token_id, 0), vocab_size - 1) for token_id in token_ids]

    # Decode predictions and references
    decoded_preds = [
        tokenizer.decode(clip_token_ids(pred), skip_special_tokens=True)
        for pred in predictions
    ]
    decoded_refs = [
        tokenizer.decode(clip_token_ids(ref), skip_special_tokens=True)
        for ref in references
    ]

    # Print decoded examples to inspect issues
    print(f'decoded_preds[0]: {decoded_preds[0]}')
    print(f'decoded_refs[0]: {decoded_refs[0]}')

    # If predictions are empty or contain only special tokens, report an issue
    if not decoded_preds[0] or decoded_preds[0] == tokenizer.pad_token:
        print(f"[WARNING] Empty prediction detected for example: {decoded_preds[0]}")

    # Compute ROUGE scores
    rouge_results = rouge.compute(
        predictions=decoded_preds, 
        references=decoded_refs, 
        use_stemmer=True
    )

    # Compute BLEU score
    bleu_results = bleu.compute(
        predictions=decoded_preds, 
        references=decoded_refs
    )

    # Combine the results into a single dictionary
    metrics = {key: rouge_results[key] * 100 for key in ["rouge1", "rouge2", "rougeL", "rougeLsum"]}
    metrics["bleu"] = bleu_results["bleu"] * 100
    
    return metrics

def create_conversation(example):
    """
    Transform the dataset into a conversational format.
    The user provides the text, and the assistant provides the summary.
    """
    # Create a conversation with user and assistant roles
    messages = [
        {"role": "user", "content": example["text"]},  # User provides the text
    ]
    # Return the conversation as a dictionary
    return {"messages": messages}

def apply_chat_template(example, tokenizer):
    """ Apply the chat template to the dataset. """
    example["text"] = tokenizer.apply_chat_template(example["messages"], tokenize=False)
    return example

# Training arguments
training_args = TrainingArguments(
    output_dir="./o",
    per_device_eval_batch_size=BATCH_SIZE,
    # bf16=True,
    # fp16_full_eval=True,
    gradient_checkpointing=True,
)
# Initialize Trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=eval_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    compute_metrics=lambda x : compute_metrics_causal_lm(x, tokenizer),
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
)

# Transform the dataset into a conversational format
eval_dataset = eval_dataset.map(create_conversation, remove_columns=["text", "summary"])

eval_dataset = eval_dataset.map(
    apply_chat_template,
    num_proc=os.cpu_count(),
    fn_kwargs={"tokenizer": tokenizer},
    remove_columns=["messages"],
    desc="Applying chat template..."
)

/home/infres/abounhar/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/infres/abounhar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/home/infres/abounhar/.local/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the

In [2]:
eval_dataset[0]

{'summary_model_name': 'Qwen/Qwen2.5-7B-Instruct-AWQ',
 'tokenizer_name': 'Qwen/Qwen2.5-7B-Instruct-AWQ',
 'dataset_source': 'Abdelkareem/wikihow-arabic-summarization',
 'sequence_length': 969,
 'text': '<|user|>\nيوجد الكثير من الأساليب التي تتعدى ممارسة الجنس لتكوني قريبة من الطرف الآخر في العلاقات السليمة. يمكنك بناء الثقة والتقارب في العلاقة من خلال التحدث والاستماع والمشاركة واحترام أفكاركما ومن خلال الخروج معا والاستمتاع بصحبة بعضكما البعض. إذا حاول شريكك إخبارك بأن الجنس هو الطريقة الوحيدة التي يمكنكما التقارب بها، فقد تحتاجين إلى التفكير فيما إذا كانت هذه العلاقة هي ما تريدينه حقا. من الممتع أن تجلسي مع شريكك وتقوما بنشاطات غير جنسية. يعد اللعب بألعاب لوحية وسيلة رائعة لإراحة شريكك وتخفيف توتره دون الحاجة لممارسة الجنس. اجلسا بجانب بعضكما البعض واستندا على بعض الوسائد وتناوبا على القراءة. يمكنكما حتى اختيار الكتب بالتناوب. يعتمد قرب جلوسكما على درجة راحتك.  إذا كنت قلقة بشأن تطور الأمور، فحاولي أن تبتعدي قليلا. يمكنك الجلوس مثلا على طرف مقابل له مع تلامس قدميكما فقط. يمكنكما قر

In [3]:
def preprocess_function(examples):
    return tokenizer(examples['text'], padding=True, truncation=True)

eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 437/437 [00:01<00:00, 296.04 examples/s]


In [4]:
eval_dataset

Dataset({
    features: ['summary_model_name', 'tokenizer_name', 'dataset_source', 'sequence_length', 'text', 'input_ids', 'attention_mask'],
    num_rows: 437
})

In [5]:
# Evaluate on test set
test_results = trainer.evaluate(eval_dataset)
print(f'[INFO] Results on test set: {test_results}')

decoded_preds[0]: <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<BackBackBackBack<<UnderUnder<UnderUnder<BackUnderBackBackBackBackBackUnderUnderUnderUnder<Under<Back<<BackUnderUnderUnderBackBackBackUnderUnderUnderBackBackBackUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderBackUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder<<UnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder<UnderInternetUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder authorisedUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnde

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: abdelazizbounhar. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


[INFO] Results on test set: {'eval_loss': 2.5254828929901123, 'eval_model_preparation_time': 0.0047, 'eval_rouge1': 31.289077212390936, 'eval_rouge2': 7.000054785325904, 'eval_rougeL': 30.247449942567627, 'eval_rougeLsum': 30.559380081566523, 'eval_bleu': 2.8362468634132427, 'eval_runtime': 119.0566, 'eval_samples_per_second': 3.671, 'eval_steps_per_second': 0.462}


In [12]:
# Evaluate on test set
test_results = trainer.evaluate(eval_dataset)
print(f'[INFO] Results on test set: {test_results}')

decoded_preds[0]: <<<<<<<<<<<<<<<<<<<<<<<<<<UnderUnderUnder<<UnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder<UnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder<UnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder<UnderUnder<<UnderUnderUnder<UnderUnderUnderUnder<UnderUnderUnderUnder<<<<<<<<<<<<<<<<<<Under<<<UnderUnder<<<Under<<<<<<<<<<<Under<Under<<<<<Under<<Under<<<<<<UnderUnder<<<<<UnderUnderUnder<<<Under<<<<UnderUnderUnderUnderUnderUnder<<UnderUnderUnderUnderUnderUnder<UnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder<<UnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderAndroid<<UnderUnderUnder<UnderUnderUnderUnderUnderUnder<UnderUnder<UnderUnder<UnderUnderUnderUnderUnd

In [6]:
# Evaluate on test set
test_results = trainer.evaluate(eval_dataset)
print(f'[INFO] Results on test set: {test_results}')

decoded_preds[0]: Once<<<Once<<<<<<<<<<<<UnderUnderAuthorUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder<UnderUnderUnderUnderUnderUnderUnderUnderUnderUnderUnder<UnderUnderUnderUnderUnder<UnderUnderUnder<Under<UnderUnderAuthorUnderUnderUnderAuthorUnderUnder<<UnderUnder<UnderUnder<<<<<<Under<Under<<<<<<<Under<<Under<<<UnderUnderUnderUnderAuthorUnder<<<<UnderUnderAuthor<<Under<<Under<UnderUnderUnderUnderUnderUnderUnderAndroid<<UnderAuthorAuthorAuthor<UnderUnderUnder<<UnderUnderAuthorUnderUnderUnderUnderUnderUnderUnderUnderUn

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: abdelazizbounhar. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


[INFO] Results on test set: {'eval_loss': 2.425283432006836, 'eval_model_preparation_time': 0.0047, 'eval_rouge1': 38.30986197413715, 'eval_rouge2': 17.798467716475482, 'eval_rougeL': 37.15017882326543, 'eval_rougeLsum': 37.75851161995967, 'eval_bleu': 3.9503918932153574, 'eval_runtime': 41.1489, 'eval_samples_per_second': 10.62, 'eval_steps_per_second': 1.337}


In [7]:
# Evaluate on test set
test_results = trainer.evaluate(eval_dataset)
print(f'[INFO] Results on test set: {test_results}')

decoded_preds[0]: |user|>
يجب العديد من الأاليب التي تطلبى حدحدودسة الت لوف م مة من شخصطرف الآخر، العلاقات،رمة،  أناء علاقةقة والواصل بين العلاقات إذا خلال تحدث معماع إلىتحدث فيتبادلترام الآكاره..اق تروج للعا فيمتاع ببعبة بعضكما البعض.  كنتاولتعك أنظهبار بك ليس أمرريقة الوحيدة ل ت أن مقرب،، ف يين إلى توقفير في إذا كان هذه هي س التي تريدينك.ا.  المهمتع أن تشعرب لل شريكك فيكونين بعملات مختلفة الجنسنسية، يمكنك هذاعب علىسي العبية أوريلة جائعة لظه النفسريكك وخفيف الضوترك. أنة لمارسة الجنس. يمكنكحرص في علىجانبه شكما البعض ومجد إلى أكسائد أوقمق و بعضفةة أو إذا أيضا أيضا أن متب التيطبعوب على إذاساعد هذادركنسسكما على نوعجة التاحةتكما إذا إذا كنت تادبي أو قج العلاقةور، فحاولولي أن تجلسقيدي عنليلا عن إذامالوس بعالا ب الأرضاكة و تقلسكدميكما.. إذا أيضا أنضاءة كتاب الكصص الممانسية أوكيضكما البعض أجل تجت اانتباه عنا عن الجنس.مثلغمما تتب تشعرمن بعض بعض بعض رغك الصةص). إذاجلسا بعا علىنا ل بعضصوما يمكنكجلسا عن أات جغنية التيحاولعلسي معلغ من شريكك.اني بمشاهدةكلوكما. يمكنك ا تقملي عن من ذلكاههة الأ كلمك..

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: abdelazizbounhar. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


[INFO] Results on test set: {'eval_loss': 2.4264333248138428, 'eval_model_preparation_time': 0.0055, 'eval_rouge1': 26.59967449402053, 'eval_rouge2': 11.045408287198025, 'eval_rougeL': 25.71808991230977, 'eval_rougeLsum': 26.235493743069043, 'eval_bleu': 3.2501948192649843, 'eval_runtime': 86.764, 'eval_samples_per_second': 5.037, 'eval_steps_per_second': 0.634}
